<a href="https://colab.research.google.com/github/akshithpt109/seismic_detection/blob/main/Output_Catalogging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Feature Extraction for Seismic Data Analysis

This block of code is responsible for extracting various features from seismic waveform data stored in miniSEED files. Here’s an overview of the process:

1. **Loading and Preprocessing Data**:
   - `load_mseed()`: Loads the miniSEED file and extracts the trace data.
   - `apply_bandpass_filter()`: Applies a bandpass filter to the trace to isolate the frequency range of interest.

2. **Feature Computation**:
   - `compute_sta_lta_with_time()`: Computes the Short-Term Average / Long-Term Average (STA/LTA) characteristic function for detecting seismic events and finds the time of the highest STA/LTA value.
   - `detect_sta_lta_events()`: Detects start and end times of seismic events using STA/LTA values.
   - `extract_sta_lta_features_with_time()`: Extracts additional STA/LTA-based features, including the maximum, mean, and variance of STA/LTA values.

3. **Time Domain Features**:
   - `extract_time_domain_features_with_max_min_time()`: Extracts features such as mean amplitude, maximum amplitude, second maximum amplitude, and RMS (Root Mean Square) amplitude. It also computes the times of these amplitude peaks.

4. **Event Duration and Triggering Features**:
   - `extract_event_duration()`: Computes the duration of seismic events based on STA/LTA triggers.
   - `detect_zcr_threshold_in_batches()`, `detect_energy_threshold_in_batches()`, `detect_amplitude_spikes_in_batches()`: Detect time events based on zero-crossing rate (ZCR), cumulative energy thresholds, and amplitude spikes across different time windows.

5. **Batch Processing**:
   - `split_into_batches()`: Splits a trace into smaller batches to analyze events over specific time windows.
   - These batches help in identifying features like energy peaks or ZCR exceedance within specific time frames.

6. **Feature Extraction for Catalog**:
   - `extract_features_for_catalog()`: Reads miniSEED files from a specified directory, processes each file to extract relevant features, and stores the results in a DataFrame. It combines features such as STA/LTA-based metrics, amplitude metrics, and event durations into a single dataset, which will be used for further analysis and model training.

These features provide a detailed characterization of the seismic data and are used to train models that predict arrival times of seismic events in new data. By identifying key metrics like amplitude peaks, energy thresholds, and STA/LTA triggers, the model can better understand and predict seismic activity.

In [ ]:
import numpy as np
import pandas as pd
from scipy.fft import fft, fftfreq
from obspy.signal.trigger import classic_sta_lta, trigger_onset
from obspy import read
from datetime import datetime
from scipy.signal import find_peaks

#Load the miniSEED file
def load_mseed(file_path):
    st = read(file_path)
    tr = st.traces[0].copy()
    return tr

#Apply a bandpass filter to the trace
def apply_bandpass_filter(trace, min_freq, max_freq):
    tr_filt = trace.copy()
    tr_filt.filter('bandpass', freqmin=min_freq, freqmax=max_freq)
    return tr_filt

#Get the arrival time from the trace
def get_arrival(tr, time):
    starttime = tr.stats.starttime.datetime
    return (time - starttime).total_seconds()

#Function to compute STA/LTA with time
def compute_sta_lta_with_time(trace, sta_len, lta_len):
    sampling_rate = trace.stats.sampling_rate
    cft = classic_sta_lta(trace.data, int(sta_len * sampling_rate), int(lta_len * sampling_rate))

    max_sta_lta = np.max(cft)
    max_sta_lta_idx = np.argmax(cft)


    max_sta_lta_time = max_sta_lta_idx / sampling_rate

    return cft, max_sta_lta, max_sta_lta_time

#Function to detect STA/LTA events
def detect_sta_lta_events(trace, sta_len, lta_len):
    thr_on = 2.0
    thr_off = 1.0
    cft, _, _ = compute_sta_lta_with_time(trace, sta_len, lta_len)
    on_off = trigger_onset(cft, thr_on, thr_off)

    if len(on_off) > 0:
        start_idx, end_idx = on_off[0]
        start_time = start_idx / trace.stats.sampling_rate
        end_time = end_idx / trace.stats.sampling_rate
        return start_time, end_time
    return None, None

#Split trace into 120-second batches
def split_into_batches(trace, batch_duration_sec=120):
    sampling_rate = trace.stats.sampling_rate
    batch_size = int(batch_duration_sec * sampling_rate)
    total_length = len(trace.data)

    #Create a list of batches
    batches = []
    for i in range(0, total_length, batch_size):
        batch_data = trace.data[i:i + batch_size]
        if len(batch_data) == batch_size:
            batch_trace = trace.copy()
            batch_trace.data = batch_data
            batch_trace.stats.starttime += i / sampling_rate
            batches.append(batch_trace)
    return batches

#Function to detect Zero-Crossing Rate (ZCR) threshold events in batches
def detect_zcr_threshold_in_batches(trace, zcr_threshold):
    batches = split_into_batches(trace)
    for batch in batches:
        zcr_exceed_time = detect_zcr_threshold_events(batch, zcr_threshold)
        if zcr_exceed_time is not None:
            return (batch.stats.starttime - trace.stats.starttime) + zcr_exceed_time
    return None

#Function to detect ZCR threshold events (individual batch)
def detect_zcr_threshold_events(trace, zcr_threshold):
    data = trace.data
    sampling_rate = trace.stats.sampling_rate
    zero_crossings = np.where(np.diff(np.sign(data)))[0]
    zcr = len(zero_crossings) / len(data)

    if zcr > zcr_threshold and len(zero_crossings) > 0:
        zcr_exceed_time = zero_crossings[0] / sampling_rate
        return zcr_exceed_time
    return None

#Function to detect when the cumulative energy exceeds a threshold in batches
def detect_energy_threshold_in_batches(trace, energy_threshold):
    batches = split_into_batches(trace)
    for batch in batches:
        energy_exceed_time = detect_energy_threshold_events(batch, energy_threshold)
        if energy_exceed_time is not None:
            return (batch.stats.starttime - trace.stats.starttime) + energy_exceed_time
    return None

#Function to detect energy threshold events (individual batch)
def detect_energy_threshold_events(trace, energy_threshold):
    data = trace.data
    cumulative_energy = np.cumsum(data**2) / np.max(np.cumsum(data**2))

    exceed_indices = np.where(cumulative_energy > energy_threshold)[0]

    if len(exceed_indices) > 0:
        energy_exceed_time = exceed_indices[0] / trace.stats.sampling_rate
        return energy_exceed_time
    return None

#Function to detect amplitude spikes in batches
def detect_amplitude_spikes_in_batches(trace, amp_threshold):
    batches = split_into_batches(trace)
    for batch in batches:
        spike_time = detect_amplitude_spikes(batch, amp_threshold)
        if spike_time is not None:
            return (batch.stats.starttime - trace.stats.starttime) + spike_time
    return None

#Function to detect amplitude spikes (individual batch)
def detect_amplitude_spikes(trace, amp_threshold):
    data = trace.data
    sampling_rate = trace.stats.sampling_rate
    spike_indices = np.where(np.abs(data) > amp_threshold)[0]

    if len(spike_indices) > 0:
        spike_time = spike_indices[0] / sampling_rate
        return spike_time
    return None

def highest_average_amplitude(mseed_file, batch_duration_sec=180):
    st = read(mseed_file)
    trace = st.traces[0].copy()
    sampling_rate = trace.stats.sampling_rate
    batch_size = int(batch_duration_sec * sampling_rate)
    total_length = len(trace.data)

    max_average_amplitude = None
    time_of_max_amplitude = None

    for i in range(0, total_length, batch_size):
        batch_data = trace.data[i:i + batch_size]
        if len(batch_data) == batch_size:
            avg_amplitude = np.mean(np.abs(batch_data))
            if max_average_amplitude is None or avg_amplitude > max_average_amplitude:
                max_average_amplitude = avg_amplitude
                time_of_max_amplitude = i / sampling_rate
    return max_average_amplitude, time_of_max_amplitude

#Function to extract STA/LTA features with time
def extract_sta_lta_features_with_time(trace, sta_len, lta_len):
    cft, max_sta_lta, max_sta_lta_time = compute_sta_lta_with_time(trace, sta_len, lta_len)
    mean_sta_lta = np.mean(cft)
    var_sta_lta = np.var(cft)
    return max_sta_lta, max_sta_lta_time, mean_sta_lta, var_sta_lta

#Function to extract amplitude features with min and max times
def extract_time_domain_features_with_max_min_time(trace):
    data = trace.data
    sampling_rate = trace.stats.sampling_rate
    mean_amp = np.mean(data)
    rms_amp = np.sqrt(np.mean(data**2))
    sorted_amplitudes = np.argsort(data)
    max_amp_idx = sorted_amplitudes[-1]
    second_max_amp_idx = sorted_amplitudes[-2]
    max_amp = data[max_amp_idx]
    second_max_amp = data[second_max_amp_idx]
    min_amp = np.min(data)
    min_amp_idx = np.argmin(data)
    max_amp_time = max_amp_idx / sampling_rate
    second_max_amp_time = second_max_amp_idx / sampling_rate
    min_amp_time = min_amp_idx / sampling_rate
    return mean_amp, max_amp, max_amp_time, second_max_amp, second_max_amp_time, min_amp, min_amp_time, rms_amp



#Function to extract event duration features
def extract_event_duration(trace, sta_len, lta_len):
    thr_on = 4.0
    thr_off = 1.5
    cft = compute_sta_lta_with_time(trace, sta_len, lta_len)[0]
    on_off = trigger_onset(cft, thr_on, thr_off)
    if len(on_off) > 0:
        event_durations = [(end - start) / trace.stats.sampling_rate for start, end in on_off]
        return np.mean(event_durations) if event_durations else 0
    return 0

#Main function to extract features from the catalog
def extract_features_for_catalog(cat_file_path, data_directory):

    features_list = []

    try:
        filename = data_directory
        mseed_file = f'{cat_file_path}.mseed'

        if load_mseed(mseed_file):
            tr = load_mseed(mseed_file)
            tr_filt = apply_bandpass_filter(tr, 0.5, 1.0)
            max_sta_lta, max_sta_lta_time, mean_sta_lta, var_sta_lta = extract_sta_lta_features_with_time(tr_filt, 0.5, 10)
            mean_amp, max_amp, max_amp_time, second_max_amp, second_max_amp_time, min_amp, min_amp_time, rms_amp = extract_time_domain_features_with_max_min_time(tr)

            zcr_threshold_time = detect_zcr_threshold_in_batches(tr_filt, zcr_threshold=0.1)
            energy_threshold_time = detect_energy_threshold_in_batches(tr_filt, energy_threshold=0.7)
            amp_spike_time = detect_amplitude_spikes_in_batches(tr_filt, amp_threshold=0.4)
            highest_avg_amplitude1, time_of_occurrence1 = highest_average_amplitude(mseed_file, batch_duration_sec=200)
            highest_avg_amplitude2, time_of_occurrence2 = highest_average_amplitude(mseed_file, batch_duration_sec=475)


                #STA/LTA event detection
            sta_lta_start_time, sta_lta_end_time = detect_sta_lta_events(tr_filt, sta_len=200, lta_len=475)


            features_list.append({
                'filename': filename,
                'max_sta_lta': max_sta_lta,
                'max_sta_lta_time': max_sta_lta_time,
                'mean_sta_lta': mean_sta_lta,
                'var_sta_lta': var_sta_lta,
                'mean_amp': mean_amp,
                'max_amp': max_amp,
                'max_amp_time': max_amp_time,
                'second_max_amp': second_max_amp,
                'second_max_amp_time': second_max_amp_time,
                'min_amp': min_amp,
                'min_amp_time': min_amp_time,
                'rms_amp': rms_amp,
                'zcr_threshold_time': zcr_threshold_time,
                'energy_threshold_time': energy_threshold_time,
                'amp_spike_time': amp_spike_time,
                'highest_avg_amplitude1': highest_avg_amplitude1,
                'time_of_occurrence1': time_of_occurrence1,
                'highest_avg_amplitude2': highest_avg_amplitude2,
                'time_of_occurrence2': time_of_occurrence2,
                'sta_lta_start_time': sta_lta_start_time,
                'sta_lta_end_time': sta_lta_end_time
            })

    except Exception as e:
        print(f'Error processing row {i}: {e}')

    return pd.DataFrame(features_list)


### Predicting Arrival Times Using KNN Model

This block of code is focused on training a K-Nearest Neighbors (KNN) model to predict the arrival times of seismic events based on extracted features from the miniSEED files. The steps involved are as follows:

1. **Load and Preprocess the Training Data**:
   - Reads the training dataset from `Features.csv` and drops any columns with missing values.
   - Selects relevant features and scales them using `StandardScaler` to ensure that all features have zero mean and unit variance.
   - Defines the target variable `y_absolute` as the 'arrival' time.
   - Splits the data into training and testing sets for model evaluation.

2. **Feature Extraction for Testing Data**:
   - Iterates over a directory containing the test miniSEED files, extracting features using the `extract_features_for_catalog()` function for each file.
   - Concatenates all the extracted features into a single DataFrame, preparing them for model prediction.

3. **Prepare Test Data for Prediction**:
   - Prepares the test features by dropping columns not used for prediction and scales them using the same scaler fitted on the training data.
   - Ensures that the feature space matches between training and test data to avoid errors during prediction.

4. **Train and Apply the KNN Model**:
   - Trains a KNN model with `n_neighbors=5` using the training dataset (`X_train_abs` and `y_train_abs`).
   - Uses the trained model to predict arrival times for the test dataset.

5. **Store and Save Predictions**:
   - Stores the predicted arrival times along with their corresponding filenames into a list of dictionaries.
   - Converts this list into a DataFrame called `output_df`, which contains the filenames and their respective predicted arrival times (`time_rel`).
   - Saves the predictions to a CSV file (`output_catalog.csv`) for further analysis or reporting.

In [ ]:
import os
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Load the initial dataset and preprocess
df = pd.read_csv("Features.csv")
df = df.dropna(axis=1)
features_df_clean = df.drop(columns=['arrival','zcr_threshold_time', 'sta_lta_start_time', 'max_sta_lta', 'mean_sta_lta', 'var_sta_lta', 'mean_amp', 'max_amp', 'second_max_amp', 'min_amp', 'rms_amp', 'highest_avg_amplitude1', 'highest_avg_amplitude2'])
scaler = StandardScaler()
features_df_scaled = scaler.fit_transform(features_df_clean)
X = features_df_scaled
y_absolute = df['arrival']
X_train_abs, X_test_abs, y_train_abs, y_test_abs = train_test_split(X, y_absolute, test_size=0.2, random_state=42)


data_dir = r'C:\Users\akshi\Machine Learning Projects\Space Apps Challenge\data\lunar\test\data'


all_features = []


for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.mseed'):

            filepath_without_extension = os.path.join(root, os.path.splitext(file)[0])


            features_df = extract_features_for_catalog(filepath_without_extension, os.path.splitext(file)[0])


            all_features.append(features_df)

#Concatenate all extracted feature data into a single DataFrame
final_dataset = pd.concat(all_features, ignore_index=True)
X_test = final_dataset.drop(columns=['filename', 'zcr_threshold_time', 'sta_lta_start_time', 'max_sta_lta', 'mean_sta_lta', 'var_sta_lta', 'mean_amp', 'max_amp', 'second_max_amp', 'min_amp', 'rms_amp', 'highest_avg_amplitude1', 'highest_avg_amplitude2']).dropna(axis=1).values

#Scale the features using the same scaler as the training data
X_test_scaled = scaler.transform(X_test)

#Train the KNN model (ensure the model is properly trained)
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X_train_abs, y_train_abs)


predictions = knn_model.predict(X_test_scaled)


results = []
for i, pred in enumerate(predictions):
    results.append({
        'filename': final_dataset['filename'].iloc[i],
        'time_rel': pred
    })


output_df = pd.DataFrame(results)
output_file = 'output_catalog.csv'
output_df.to_csv(output_file, index=False)
print(f"Predictions saved to {output_file}")

                                 filename    time_rel
0  xa.s12.00.mhz.1969-12-16HR00_evid00006   7583.5416
1  xa.s12.00.mhz.1970-01-09HR00_evid00007  55019.5094
2  xa.s12.00.mhz.1970-02-07HR00_evid00014  68063.6764
3  xa.s12.00.mhz.1970-02-18HR00_evid00016  51983.6214
4  xa.s12.00.mhz.1970-03-14HR00_evid00018  36275.6548
Predictions saved to output_catalog.csv


c:\Users\akshi\Machine Learning Projects\Space Apps Challenge\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
